In [1]:
import sys
#sys.path.append("fashion-clip/")
from fashion_clip.fashion_clip import FashionCLIP
import numpy as np
from sklearn.metrics import *
import pandas as pd

In [2]:
# %%capture
fclip = FashionCLIP('fashion-clip')

In [3]:
annotations = pd.read_csv('D:\M1/fashion/lda_model/data/anotation_new.csv')
img_path = annotations.iloc[:, 0]
images = img_path.values[:100]

In [4]:
item_set = set()
new_images = []
for i in images:
    item_id = i.split('/')[-1]
    if item_id in item_set:
        continue
    new_images.append(i)
    item_set.add(item_id)
images = new_images
images = [i.replace('\\', '/') for i in images]
image_embeddings = fclip.encode_images(images, batch_size=128)
image_embeddings = image_embeddings/np.linalg.norm(image_embeddings, ord=2, axis=-1, keepdims=True)
category = ['Vest_top', 'Hair/alice_band', 'Leggings/Tights', 'T-shirt', 'Sneakers',
 'Sunglasses', 'Cardigan', 'Gloves', 'Underwear_Tights', 'Hoodie', 'Other_shoe',
 'Shorts' ,'Jumpsuit/Playsuit', 'Dress', 'Trousers', 'Belt', 'Socks',
 'Underwear_bottom' ,'Bodysuit' ,'Hat/beanie', 'Scarf' ,'Jacket',
 'Other_accessories', 'Bra' ,'Swimwear_bottom' ,'Blazer' ,'Top' ,'Polo shirt',
 'Sweater' ,'Necklace' ,'Pyjama_set' ,'Blouse', 'Bag', 'Shirt' ,'Coat' ,'Boots',
 'Skirt', 'Garment_Set', 'Bikini_top', 'Sandals' ,'Dungarees', 'Earring',
 'Cap/peaked' ,'Ballerinas', 'Swimsuit' ,'Hat/brim']
materials = ['Cotton', 'Polyester', 'Rayon', 'Linen', 'Wool', 'Silk', 'Nylon', 'Polyurethane', 'Denim', 'Spandex']
colors = ['Black', 'White', 'Gray', 'Navy', 'Blue', 'Red', 'Green', 'Brown', 'Beige', 'Pink',
          'Yellow', 'Orange', 'Purple', 'Lavender', 'Teal', 'Turquoise', 'Magenta', 'Olive', 'Maroon', 'Charcoal']
created_material_labels = []
created_color_labels = []
for c in category:
    for color in colors:
        created_color_labels.append(f'{c}_{color}')
    for m in materials:
        created_material_labels.append(f'{c}_{m}')

Parameter 'transform'=<function FashionCLIP.encode_images.<locals>.transform_fn at 0x000001752044E310> of the transform datasets.arrow_dataset.Dataset.set_format couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
1it [00:17, 17.12s/it]


In [ ]:
label_material_embeddings = fclip.encode_text(created_material_labels, batch_size=32)
label_material_embeddings = label_material_embeddings/np.linalg.norm(label_material_embeddings, ord=2, axis=-1, keepdims=True)
label_color_embeddings = fclip.encode_text(created_color_labels, batch_size=32)
label_color_embeddings = label_color_embeddings/np.linalg.norm(label_color_embeddings, ord=2, axis=-1, keepdims=True)

15it [00:01, 12.23it/s]                        
29it [00:02, 12.89it/s]                        


In [ ]:
# predicteds
predicted_materials = label_material_embeddings.dot(image_embeddings.T)
predicted_materials = [created_material_labels[k] for k in np.argmax(predicted_materials, axis=0)]
predicted_colors = label_color_embeddings.dot(image_embeddings.T)
predicted_colors = [created_color_labels[k] for k in np.argmax(predicted_colors, axis=0)]

In [ ]:
with open('D:\M1/fashion/lda_model/data/fashion_clip_new.txt', 'w') as f:
    for img_path, m, c in zip(images, predicted_materials, predicted_colors):
        # item_id = i.split('/')[-1]
        item_id = img_path.split('/')[-1][:-6]
        f.write(f'{item_id}, {m}, {c} \n')